# API OpenAI - Dados de posts manifestações

In [1]:
# imports
from openai import OpenAI
import os
import pandas as pd
from groq import Groq
import openpyxl

In [ ]:
# api key
client = Groq(api_key="MyAPIKey")

In [4]:
# alterar PATH para o caminho do arquivo Posts_2013.xlsx
# data = pd.read_excel(r'G:\Drives compartilhados\LEMEP-Manchetometro\Livro Junho 2013\Textos Jornais 2013\Capas\FSP_2003-2013\manchetes_FSP_agregado_4.0.xlsx')
data = pd.read_excel("/Users/joaoferes/Library/CloudStorage/GoogleDrive-jferes@iesp.uerj.br/Shared drives/LEMEP-Manchetometro/Livro Junho 2013/Textos Jornais 2013/Capas/FSP_2003-2013/manchetes_FSP_agregado.xlsx")
data.head(10)

,Arquivo,Data,Manchete
0,FSP_2010-01-01_0001.jpg,2010-01-01,Chuva mata ao menos 19 no RJ
1,FSP_2010-01-02_0001.jpg,2010-01-02,Soterraamentos matam 30 em Angra
2,FSP_2010-01-03_0001.jpg,2010-01-03,Empresários vão priorizar doações ocultas na e...
3,FSP_2010-01-04_0001.jpg,2010-01-04,Inaugurações dominam ano eleitoral
4,FSP_2010-01-05_0001.jpg,2010-01-05,FAB prefere caça sueco a francês
5,FSP_2010-01-06_0001.jpg,2010-01-06,Ponte desaba em rodovia do Sul; dez pessoas de...
6,FSP_2010-01-07_0001.jpg,2010-01-07,"Prejuízo com chuva supera R$ 1,1 bi"
7,FSP_2010-01-08_0001.jpg,2010-01-08,Jobim fará seu próprio relatório sobre os acags
8,FSP_2010-01-09_0001.jpg,2010-01-09,Ministro critica plano de direitos humanos
9,FSP_2010-01-10_0001.jpg,2010-01-10,Vannuchi ameaça demissão se plano punir tortur...


In [5]:
len(data)

3866

In [6]:
# funcao para usar o modelo preenchendo respostas
def use_model(question, text):
    completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role":"system", "content":"Você é um assistente que analisa manchetes de jornal e responde perguntas a respeito."},
        {"role":"user", "content":f"Manchete:{text}\n\nPergunta:{question}"}
    ]
)
    return completion.choices[0].message.content

In [7]:
# adicionar colunas novas (inicialmente vazias) para as respostas
data["Menção ao Lula"] = "" # cada variável adicionada modifica o valor entre colchetes
data["Valência Lula"] = ""
data

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula
0,FSP_2010-01-01_0001.jpg,2010-01-01,Chuva mata ao menos 19 no RJ,,
1,FSP_2010-01-02_0001.jpg,2010-01-02,Soterraamentos matam 30 em Angra,,
2,FSP_2010-01-03_0001.jpg,2010-01-03,Empresários vão priorizar doações ocultas na e...,,
3,FSP_2010-01-04_0001.jpg,2010-01-04,Inaugurações dominam ano eleitoral,,
4,FSP_2010-01-05_0001.jpg,2010-01-05,FAB prefere caça sueco a francês,,
...,...,...,...,...,...
3861,FSP_2011-12-27_0001.jpg,2011-12-27,Homicídio cai e roubo de veículos sobe em SP,,
3862,FSP_2011-12-28_0001.jpg,2011-12-28,Faculdades cobram R$ 500 milhões de bolsa do g...,,
3863,FSP_2011-12-29_0001.jpg,2011-12-29,Tráfego para o interior de SP sobe mais que pa...,,
3864,FSP_2011-12-30_0001.jpg,2011-12-30,"Em 2011, ouro e dólar foram os mais rentáveis",,


In [8]:
# definindo as perguntas 1 e 2

pergunta_1 = """A manchete menciona o Lula?

Responda com apenas "Sim" ou "Não", sem qualquer texto adicional."""

pergunta_2 = """Qual é a valência da menção ao Lula?

Responda com apenas "Favorável", "Contrária" ou "Neutra", sem qualquer texto adicional.

Sendo que:
- Favorável: a menção ao Lula é posita, citando dados ou fatos positivos sobre ele ou seu governo. 
- Contrária: a menção ao Lula é negativa, citando dados ou fatos negativos sobre ele ou seu governo.
- Neutra: a menção ao Lula não é nem positiva nem negativa, apenas informativa.
"""

In [9]:
# for cada linha no dataset
    # montando uma string com todas as informacoes importantes do post que queremos passar pra ia 
    # usando a funcao use_model para responder a pergunta no post atual
    # adicionando a resposta no dataset

for index, row in data.iterrows():    
    if index <= 3559: 
        continue
    manchete = row["Manchete"]
    # usar o modelo para responder a pergunta
    answer_1 = use_model(pergunta_1, manchete)
    data.loc[index, "Menção ao Lula"] = answer_1 # precisa mudar todas vez que for adicionar variável nova
    # answer_2 = use_model(pergunta_2, manchete)
    # data.loc[index, "Valência Lula"] = answer_2 # precisa mudar todas vez que for adicionar variável nova
    # if index == 100:
    #     break
    
    print(f"Index: {index}", f"Manchete: {manchete}", f"Resposta 1: {answer_1}")

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jj1xhsq4fqb8y3zq3k5jk9vs` service tier `on_demand` on requests per minute (RPM): Limit 30, Used 30, Requested 1. Please try again in 1.659s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'requests', 'code': 'rate_limit_exceeded'}}

In [10]:
data

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula,Governo Federal,Valência Governo Federal
0,ESP_2003-01-01_0001.jpg,01/01/2003,Festa e esperança para Lula presidente,Sim,Favorável,Não,NaN
1,ESP_2003-01-02_0001.jpg,02/01/2003,"Vamos mudar, sim. Mudar com coragem e com cuidado",Sim,Favorável,Não,NaN
2,ESP_2003-01-03_0001.jpg,03/01/2003,Palocci promete crescimento sem inflação,Não,NaN,Não,NaN
3,ESP_2003-01-04_0001.jpg,04/01/2003,Governo vai mandar no preço dos combustíveis,Não,NaN,Sim,Neutra
4,ESP_2003-01-05_0001.jpg,05/01/2003,"Reajuste de preços públicos não muda, garante ...",Não,NaN,Sim,Favorável
...,...,...,...,...,...,...,...
4013,ESP_2013-12-27_0001.jpg,27/12/2013,"Com crédito contido e juros altos, vendas de N...",Não,NaN,Não,NaN
4014,ESP_2013-12-28_0001.jpg,28/12/2013,"Com deságio recorde, fundos de pensão e OAS le...",Não,NaN,Não,NaN
4015,ESP_2013-12-29_0001.jpg,29/12/2013,Gasto com servidor cresce mais que receita em ...,Não,NaN,Não,NaN
4016,ESP_2013-12-30_0001.jpg,30/12/2013,"Na TV, Dilma diz que ‘guerra psicológica’ inib...",Não,NaN,Sim,Contrária


In [29]:
data_mencao_Lula = data[data["Menção ao Lula"] == "Sim"] # precisa mudar todas vez que for adicionar variável nova
data_mencao_Lula

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula
0,ESP_2003-01-01_0001.jpg,01/01/2003,Festa e esperança para Lula presidente,Sim,Favorável
1,ESP_2003-01-02_0001.jpg,02/01/2003,"Vamos mudar, sim. Mudar com coragem e com cuidado",Sim,Favorável
17,ESP_2003-01-18_0001.jpg,18/01/2003,"Chávez, de surpresa, decide vir ao Brasil enco...",Sim,Neutra
18,ESP_2003-01-19_0001.jpg,19/01/2003,Lula diz não a Chávez e mantém Grupo de Amigos,Sim,Neutra
22,ESP_2003-01-23_0001.jpg,23/01/2003,Primeiro Copom do governo Lula eleva juros,Sim,Neutra
...,...,...,...,...,...
3748,ESP_2013-04-06_0001.jpg,06/04/2013,MPF abre inquérito para investigar Lula no men...,Sim,
3766,ESP_2013-04-24_0001.jpg,24/04/2013,PF vai pedir quebra de sigilo de assessor de Lula,Sim,
3782,ESP_2013-05-10_0001.jpg,10/05/2013,Dilma e Lula veem articulação política frágil ...,Sim,
3852,ESP_2013-07-19_0001.jpg,19/07/2013,Ibope: Lula supera Dilma em votos,Sim,


In [31]:
for index, row in data_mencao_Lula.iterrows():    
    manchete = row["Manchete"]
    
    # usar o modelo para responder a pergunta
    answer_2 = use_model(pergunta_2, manchete)
    data.loc[index, "Valência Lula"] = answer_2 # precisa mudar todas vez que for adicionar variável nova
    # if index == 100:
    #     break
    
    print(f"Index: {index}", f"Manchete: {manchete}", f"Resposta: {answer_2}")


Index: 0 Manchete: Festa e esperança para Lula presidente Resposta: Favorável
Index: 1 Manchete: Vamos mudar, sim. Mudar com coragem e com cuidado Resposta: Favorável
Index: 17 Manchete: Chávez, de surpresa, decide vir ao Brasil encontrar Lula Resposta: Neutra
Index: 18 Manchete: Lula diz não a Chávez e mantém Grupo de Amigos Resposta: Neutra
Index: 22 Manchete: Primeiro Copom do governo Lula eleva juros Resposta: Neutra
Index: 25 Manchete: Lula apoia França e Alemanha contra o ataque Resposta: Neutra
Index: 28 Manchete: Para FMI, Lula merece apoio dos mercados Resposta: Favorável
Index: 48 Manchete: "Vim propor a paz", diz Lula ao Congresso Resposta: Favorável
Index: 51 Manchete: Lula dirá aos governadores que sem as reformas os Estados quebram Resposta: Neutra
Index: 56 Manchete: Economia só muda após as reformas, avisa Lula Resposta: Neutra
Index: 58 Manchete: Lula autoriza Forças Armadas nas ruas do Rio Resposta: Neutra
Index: 69 Manchete: Lula vai antecipar envio de reformas Respo

In [10]:
# Inserindo as variáveis Governo Federal e Valência Governo Federal
# adicionar colunas novas (inicialmente vazias) para as respostas
data["Governo Federal"] = "" # cada variável adicionada modifica o valor entre colchetes
data["Valência Governo Federal"] = ""
data

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula,Governo Federal,Valência Governo Federal
0,ESP_2003-01-01_0001.jpg,01/01/2003,Festa e esperança para Lula presidente,Sim,Favorável,,
1,ESP_2003-01-02_0001.jpg,02/01/2003,"Vamos mudar, sim. Mudar com coragem e com cuidado",Sim,Favorável,,
2,ESP_2003-01-03_0001.jpg,03/01/2003,Palocci promete crescimento sem inflação,Não,NaN,,
3,ESP_2003-01-04_0001.jpg,04/01/2003,Governo vai mandar no preço dos combustíveis,Não,NaN,,
4,ESP_2003-01-05_0001.jpg,05/01/2003,"Reajuste de preços públicos não muda, garante ...",Não,NaN,,
...,...,...,...,...,...,...,...
4013,ESP_2013-12-27_0001.jpg,27/12/2013,"Com crédito contido e juros altos, vendas de N...",Não,NaN,,
4014,ESP_2013-12-28_0001.jpg,28/12/2013,"Com deságio recorde, fundos de pensão e OAS le...",Não,NaN,,
4015,ESP_2013-12-29_0001.jpg,29/12/2013,Gasto com servidor cresce mais que receita em ...,Não,NaN,,
4016,ESP_2013-12-30_0001.jpg,30/12/2013,"Na TV, Dilma diz que ‘guerra psicológica’ inib...",Não,NaN,,


In [ ]:
# definindo as perguntas 3 e 4

pergunta_3 = """A manchete menciona o Governo Federal?

Responda com apenas "Sim" ou "Não", sem qualquer texto adicional.
Menções à palavra Governo devem ser consideradas positivas, mesmo que não utilizem o adjetivo "Federal"."""

pergunta_4 = """Qual é a valência da menção ao Governo Federal?

Responda com apenas "Favorável", "Contrária" ou "Neutra", sem qualquer texto adicional.

Sendo que:
- Favorável: a menção ao Governo Federal é posita, citando dados ou fatos positivos sobre seus resultados e políticas públicas. 
- Contrária: a menção ao Governo Federal é negativa, citando dados ou fatos negativos sobre seus resultados e políticas públicas.
- Neutra: a menção ao Governo Federal não é nem positiva nem negativa, apenas informativa.
"""

In [ ]:
# for cada linha no dataset
    # montando uma string com todas as informacoes importantes do post que queremos passar pra ia 
    # usando a funcao use_model para responder a pergunta no post atual
    # adicionando a resposta no dataset

for index, row in data.iterrows():    
    # if index <= 2288: 
    #    continue
    manchete = row["Manchete"]
    # usar o modelo para responder a pergunta
    answer_3 = use_model(pergunta_3, manchete)
    data.loc[index, "Governo Federal"] = answer_3 # precisa mudar todas vez que for adicionar variável nova
    # if index == 100:
    #     break
    
    print(f"Index: {index}", f"Manchete: {manchete}", f"Resposta 3: {answer_3}")

In [14]:
data_mencao_GF = data[data["Governo Federal"] == "Sim"] 
data_mencao_GF

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula,Governo Federal,Valência Governo Federal
3,ESP_2003-01-04_0001.jpg,04/01/2003,Governo vai mandar no preço dos combustíveis,Não,NaN,Sim,
4,ESP_2003-01-05_0001.jpg,05/01/2003,"Reajuste de preços públicos não muda, garante ...",Não,NaN,Sim,
7,ESP_2003-01-08_0001.jpg,08/01/2003,Governo manda Estados cumprirem Lei Fiscal,Não,NaN,Sim,
10,ESP_2003-01-11_0001.jpg,11/01/2003,Palocci: 'Vamos cortar todo que não seja urgen...,Não,NaN,Sim,
12,ESP_2003-01-13_0001.jpg,13/01/2003,Meta de superávit primário pode crescer,Não,NaN,Sim,
...,...,...,...,...,...,...,...
4003,ESP_2013-12-17_0001.jpg,17/12/2013,Governo contesta decisão do TCU que barra leil...,Não,NaN,Sim,
4004,ESP_2013-12-18_0001.jpg,18/12/2013,Governo recua e mantém prazo para ABS e airbag,Não,NaN,Sim,
4010,ESP_2013-12-24_0001.jpg,24/12/2013,BR-040 atrai 8 grupos; governo já analisa nova...,Não,NaN,Sim,
4016,ESP_2013-12-30_0001.jpg,30/12/2013,"Na TV, Dilma diz que ‘guerra psicológica’ inib...",Não,NaN,Sim,


In [16]:
for index, row in data_mencao_GF.iterrows():    
    manchete = row["Manchete"]
    
    # usar o modelo para responder a pergunta
    answer_4 = use_model(pergunta_4, manchete)
    data.loc[index, "Valência Governo Federal"] = answer_4 # precisa mudar todas vez que for adicionar variável nova
    # if index == 100:
    #     break
    
    print(f"Index: {index}", f"Manchete: {manchete}", f"Resposta: {answer_4}")


Index: 3 Manchete: Governo vai mandar no preço dos combustíveis Resposta: Neutra
Index: 4 Manchete: Reajuste de preços públicos não muda, garante Palocci Resposta: Favorável
Index: 7 Manchete: Governo manda Estados cumprirem Lei Fiscal Resposta: Neutra
Index: 10 Manchete: Palocci: 'Vamos cortar todo que não seja urgente socialmente' Resposta: Favorável
Index: 12 Manchete: Meta de superávit primário pode crescer Resposta: Neutra
Index: 15 Manchete: Mello abala mercados; ministros defendem o fim dos privilégios Resposta: Contrária
Index: 22 Manchete: Primeiro Copom do governo Lula eleva juros Resposta: Neutra
Index: 34 Manchete: Orçamento de Bush prevê um déficit histórico de US$ 304 bi Resposta: Contrária
Index: 37 Manchete: Governo quer arquivar a mudança da CLT Resposta: Contrária
Index: 41 Manchete: Governo corta R$ 14 bi para garantir superavit Resposta: Neutra
Index: 42 Manchete: Corte atinge todos os ministérios e reduz em 80% os investimentos Resposta: Contrária
Index: 47 Manchet

In [11]:
data["PT"] = "" # cada variável adicionada modifica o valor entre colchetes
data["Valência PT"] = ""
data

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula,Governo Federal,Valência Governo Federal,PT,Valência PT
0,ESP_2003-01-01_0001.jpg,01/01/2003,Festa e esperança para Lula presidente,Sim,Favorável,Não,NaN,,
1,ESP_2003-01-02_0001.jpg,02/01/2003,"Vamos mudar, sim. Mudar com coragem e com cuidado",Sim,Favorável,Não,NaN,,
2,ESP_2003-01-03_0001.jpg,03/01/2003,Palocci promete crescimento sem inflação,Não,NaN,Não,NaN,,
3,ESP_2003-01-04_0001.jpg,04/01/2003,Governo vai mandar no preço dos combustíveis,Não,NaN,Sim,Neutra,,
4,ESP_2003-01-05_0001.jpg,05/01/2003,"Reajuste de preços públicos não muda, garante ...",Não,NaN,Sim,Favorável,,
...,...,...,...,...,...,...,...,...,...
4013,ESP_2013-12-27_0001.jpg,27/12/2013,"Com crédito contido e juros altos, vendas de N...",Não,NaN,Não,NaN,,
4014,ESP_2013-12-28_0001.jpg,28/12/2013,"Com deságio recorde, fundos de pensão e OAS le...",Não,NaN,Não,NaN,,
4015,ESP_2013-12-29_0001.jpg,29/12/2013,Gasto com servidor cresce mais que receita em ...,Não,NaN,Não,NaN,,
4016,ESP_2013-12-30_0001.jpg,30/12/2013,"Na TV, Dilma diz que ‘guerra psicológica’ inib...",Não,NaN,Sim,Contrária,,


In [15]:
# definindo as perguntas 5 e 6

pergunta_5 = """A manchete menciona o PT?

Responda com apenas "Sim" ou "Não", sem qualquer texto adicional.
Menções a pessoas importantes do partido, como Lula, Dilma ou outros deve ser consideradas como menções ao PT."""

pergunta_6 = """Qual é a valência da menção ao PT?

Responda com apenas "Favorável", "Contrária" ou "Neutra", sem qualquer texto adicional.

Sendo que:
- Favorável: a menção ao PT é posita, citando dados ou fatos positivos sobre seus resultados e políticas públicas. 
- Contrária: a menção ao PT é negativa, citando dados ou fatos negativos sobre seus resultados e políticas públicas.
- Neutra: a menção ao PT não é nem positiva nem negativa, apenas informativa.
"""

In [ ]:
# Rodando código PT (pergunta 5)

for index, row in data.iterrows():    
#   if index <= 1902: 
#        continue
    manchete = row["Manchete"]
    # usar o modelo para responder a pergunta
    answer_5 = use_model(pergunta_5, manchete)
    data.loc[index, "PT"] = answer_5 # precisa mudar todas vez que for adicionar variável nova
    # if index == 100:
    #     break
    
    print(f"Index: {index}", f"Manchete: {manchete}", f"Resposta 5: {answer_5}")

Index: 0 Manchete: Festa e esperança para Lula presidente Resposta 5: Sim
Index: 1 Manchete: Vamos mudar, sim. Mudar com coragem e com cuidado Resposta 5: Não
Index: 2 Manchete: Palocci promete crescimento sem inflação Resposta 5: Não
Index: 3 Manchete: Governo vai mandar no preço dos combustíveis Resposta 5: Não
Index: 4 Manchete: Reajuste de preços públicos não muda, garante Palocci Resposta 5: Não
Index: 5 Manchete: Duplo ataque suicida mata 23 e fere 100 em Tel-Aviv Resposta 5: Não
Index: 6 Manchete: Confiança externa aumenta, derruba dólar e anima Bolsa Resposta 5: Não
Index: 7 Manchete: Governo manda Estados cumprirem Lei Fiscal Resposta 5: Não
Index: 8 Manchete: Fracassa de novo acordo entre o PT e o PMDB Resposta 5: Sim
Index: 9 Manchete: PT define regra para reforma da Previdência Resposta 5: Sim
Index: 10 Manchete: Palocci: 'Vamos cortar todo que não seja urgente socialmente' Resposta 5: Não
Index: 11 Manchete: Estados são obstáculo à reforma da Previdência Resposta 5: Não
In

In [9]:
data

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula,Governo Federal,Valência Governo Federal,PT,Valência PT
0,ESP_2003-01-01_0001.jpg,01/01/2003,Festa e esperança para Lula presidente,Sim,Favorável,Sim,NaN,,
1,ESP_2003-01-02_0001.jpg,02/01/2003,"Vamos mudar, sim. Mudar com coragem e com cuidado",Sim,Favorável,Não,NaN,,
2,ESP_2003-01-03_0001.jpg,03/01/2003,Palocci promete crescimento sem inflação,Não,NaN,Não,NaN,,
3,ESP_2003-01-04_0001.jpg,04/01/2003,Governo vai mandar no preço dos combustíveis,Não,NaN,Não,Neutra,,
4,ESP_2003-01-05_0001.jpg,05/01/2003,"Reajuste de preços públicos não muda, garante ...",Não,NaN,Não,Favorável,,
...,...,...,...,...,...,...,...,...,...
4013,ESP_2013-12-27_0001.jpg,27/12/2013,"Com crédito contido e juros altos, vendas de N...",Não,NaN,Não,NaN,,
4014,ESP_2013-12-28_0001.jpg,28/12/2013,"Com deságio recorde, fundos de pensão e OAS le...",Não,NaN,Não,NaN,,
4015,ESP_2013-12-29_0001.jpg,29/12/2013,Gasto com servidor cresce mais que receita em ...,Não,NaN,Não,NaN,,
4016,ESP_2013-12-30_0001.jpg,30/12/2013,"Na TV, Dilma diz que ‘guerra psicológica’ inib...",Não,NaN,Sim,Contrária,,


In [ ]:
data.to_csv(r'G:\Drives compartilhados\LEMEP-Manchetometro\Livro Junho 2013\Textos Jornais 2013\Capas\ESP_2003-2013\manchetes_ESP_teste_4.csv', sep=';', index=False) # cuidado para não sobreescrever

In [17]:
data.to_excel(r'G:\Drives compartilhados\LEMEP-Manchetometro\Livro Junho 2013\Textos Jornais 2013\Capas\ESP_2003-2013\manchetes_ESP_agregado_4.0.xlsx', index=False)